In [1]:
# !sudo apt install swig

In [2]:
# !pip install quantstats
# !pip install scikit-learn
# !pip install torch
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

# Initializing

In [1]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True
import torch

import numpy as np
import pandas as pd

from sklearn.preprocessing import MaxAbsScaler
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from utils import PortfolioOptimizationEnv, custom_reward_function, sharpe_ratio_reward_function
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Data Import and Preprocessing

In [2]:
TEST_SET = [
    "AAPL", "CVX", "GS", "JNJ",
    "JPM", "MSFT", "PFE", "PG",
    "GOOG", "XOM"
]
START_DATE = '2011-01-01'
END_DATE = '2023-12-31'
print(len(TEST_SET))

##COVARIATE 1: PRICES

portfolio_raw_df = YahooDownloader(start_date = START_DATE,
                                end_date = END_DATE,
                                ticker_list = TEST_SET).fetch_data()

10
[*********************100%%**********************]  1 of 1 completed

C:\Users\xutia\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\xutia\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

C:\Users\xutia\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\xutia\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed

...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

C:\Users\xutia\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\xutia\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



[*********************100%%**********************]  1 of 1 completed

C:\Users\xutia\AppData\Roaming\Python\Python310\site-packages\yfinance\utils.py:771: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



Shape of DataFrame:  (32700, 8)


In [5]:
##COVARIATES 2,3: VIX, FEAR/GREED INDEX

combined_spy_vix_fear_greed = pd.read_csv('datasets/VIX_feargreed/spy_vix_fear_greed_2011_2023.csv')


portfolio_raw_df['date'] = pd.to_datetime(portfolio_raw_df['date'])
combined_spy_vix_fear_greed['Date'] = pd.to_datetime(combined_spy_vix_fear_greed['Date'])


df_portfolio_comb = pd.merge(portfolio_raw_df, combined_spy_vix_fear_greed, left_on='date', right_on='Date', how='left')


df_portfolio_comb = df_portfolio_comb.drop(columns=['Date'])
df_portfolio_comb['date'] = df_portfolio_comb['date'].dt.strftime('%Y-%m-%d')

In [6]:
##COVARIATE 4: STOCK HISTORICAL RATINGS

FMP_historical_ratings = pd.read_csv('datasets/historical_ratings/FMP_historical_ratings.csv')

df_portfolio_comb_2 = df_portfolio_comb.merge(
    FMP_historical_ratings[['date', 'symbol', 'ratingScore', 'ratingDetailsDCFScore', 'ratingDetailsROEScore', 'ratingDetailsROAScore', 'ratingDetailsPEScore', 'ratingDetailsPBScore']],
    left_on=['date', 'tic'],
    right_on=['date', 'symbol'],
    how='left' )

df_portfolio_comb_2.drop(columns=['symbol'], inplace=True)

In [7]:
##COVARIATE 5: Macro Indicators

expenses = pd.read_csv('datasets/macro_indicators/expenses.csv')
gdp = pd.read_csv('datasets/macro_indicators/gdp.csv')
goods_services = pd.read_csv('datasets/macro_indicators/goods_and_services.csv')
pound_dollar = pd.read_csv('datasets/macro_indicators/pound-dollar-exchange-rate-historical-chart.csv')
unemployment = pd.read_csv('datasets/macro_indicators/unemployment.csv')
index_consumer_services = pd.read_csv('datasets/macro_indicators/index_consumer_services.csv')

df_portfolio_comb_2['date'] = pd.to_datetime(df_portfolio_comb_2['date'])
expenses['date'] = pd.to_datetime(unemployment['date'])
gdp['date'] = pd.to_datetime(gdp['DATE'])
goods_services['date'] = pd.to_datetime(goods_services['date'])
pound_dollar['date'] = pd.to_datetime(pound_dollar['date'])
unemployment['date'] = pd.to_datetime(unemployment['date'])
index_consumer_services['date'] = pd.to_datetime(index_consumer_services['date'])

df_portfolio_comb_3 = pd.merge(df_portfolio_comb_2, expenses, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, gdp, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, goods_services, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, pound_dollar, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, unemployment, on='date', how='left')
df_portfolio_comb_3 = pd.merge(df_portfolio_comb_3, index_consumer_services, on='date', how='left')

# Sort the dataframe by date (and optionally 'tic' if needed)
df_portfolio_comb_3 = df_portfolio_comb_3.sort_values(by=['date', 'tic'])

# Forward fill missing data
df_portfolio_comb_3['expenses'] = df_portfolio_comb_3['expenses'].ffill()
df_portfolio_comb_3['GDP'] = df_portfolio_comb_3['GDP'].ffill()
df_portfolio_comb_3['exports'] = df_portfolio_comb_3['exports'].ffill()
df_portfolio_comb_3['imports'] = df_portfolio_comb_3['imports'].ffill()
df_portfolio_comb_3['pound_dollar_exchange_rate'] = df_portfolio_comb_3['pound_dollar_exchange_rate'].ffill()
df_portfolio_comb_3['unemployment'] = df_portfolio_comb_3['unemployment'].ffill()
df_portfolio_comb_3['ics'] = df_portfolio_comb_3['ics'].ffill()
df_portfolio_comb_3.drop(columns=['DATE'], inplace=True)

In [16]:
##COVARIATE 6: Fundamental Indicators

DBITDA_ratio = pd.read_csv('datasets/fundemental indicators/DBITDA_ratio.csv')
PB_ratio = pd.read_csv('datasets/fundemental indicators/PB_ratio.csv')
PE_ratio = pd.read_csv('datasets/fundemental indicators/PE_ratio.csv')
PS_ratio = pd.read_csv('datasets/fundemental indicators/PS_ratio.csv')
quarterly_data = pd.read_csv('datasets/fundemental indicators/quaterlydata.csv')
ROE = pd.read_csv('datasets/fundemental indicators/ROE.csv')

df_portfolio_comb_3['date'] = pd.to_datetime(df_portfolio_comb_3['date'])
DBITDA_ratio['date'] = pd.to_datetime(DBITDA_ratio['date'])
PB_ratio['date'] = pd.to_datetime(PB_ratio['date'])
PE_ratio['date'] = pd.to_datetime(PE_ratio['date'])
PS_ratio['date'] = pd.to_datetime(PS_ratio['date'])
quarterly_data['date'] = pd.to_datetime(quarterly_data['date'])
ROE['date'] = pd.to_datetime(ROE['date'])

df_portfolio_comb_4 = df_portfolio_comb_3

df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, DBITDA_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, PB_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker','Stock Price'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, PE_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker','Stock Price'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, PS_ratio, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker','Stock Price'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, quarterly_data, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker'])
df_portfolio_comb_4 = pd.merge(df_portfolio_comb_4, ROE, left_on=['tic', 'date'], right_on=['ticker', 'date'], how='left')
df_portfolio_comb_4 = df_portfolio_comb_4.drop(columns=['ticker'])

# Sort the dataframe by date (and optionally 'tic' if needed)
df_portfolio_comb_4 = df_portfolio_comb_4.sort_values(by=['tic', 'date'])

# Forward fill missing data
df_portfolio_comb_4['TTM Revenue(Billion)'] = df_portfolio_comb_4['TTM Revenue(Billion)'].ffill()
df_portfolio_comb_4['TTM EBITDA(Billion)'] = df_portfolio_comb_4['TTM EBITDA(Billion)'].ffill()
df_portfolio_comb_4['EBITDA Margin'] = df_portfolio_comb_4['EBITDA Margin'].ffill()
df_portfolio_comb_4['Price to Book Ratio'] = df_portfolio_comb_4['Price to Book Ratio'].ffill()
df_portfolio_comb_4['PE Ratio'] = df_portfolio_comb_4['PE Ratio'].ffill()
df_portfolio_comb_4['Price to Sales Ratio'] = df_portfolio_comb_4['Price to Sales Ratio'].ffill()
df_portfolio_comb_4['Assets'] = df_portfolio_comb_4['Assets'].ffill()
df_portfolio_comb_4['NetIncomeLoss'] = df_portfolio_comb_4['NetIncomeLoss'].ffill()
df_portfolio_comb_4['Return on Equity(%)'] = df_portfolio_comb_4['Return on Equity(%)'].ffill()

C:\Users\xutia\AppData\Local\Temp\ipykernel_52124\1570567354.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  DBITDA_ratio['date'] = pd.to_datetime(DBITDA_ratio['date'])
C:\Users\xutia\AppData\Local\Temp\ipykernel_52124\1570567354.py:12: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  PB_ratio['date'] = pd.to_datetime(PB_ratio['date'])
C:\Users\xutia\AppData\Local\Temp\ipykernel_52124\1570567354.py:13: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  PE_ratio['date'] = pd.to_datetime(PE_ratio['date'])
C:\Users\xutia\AppData\Local\Temp\ipykernel_52124\1570567354.py:14: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the defa

In [9]:
# ## VMD
# from vmdpy import VMD

# def apply_vmd_to_ticker(df, alpha=5000, tau=0, K=3, DC=0, init=1, tol=1e-7):
#     """Applies VMD to the 'close' price for each ticker in the dataframe
#     and handles any length mismatch issues."""
#     vmd_features = []
    
#     # Apply VMD per ticker
#     for tic in df['tic'].unique():
#         tic_df = df[df['tic'] == tic].copy()  # Filter data for the current ticker
#         close_prices = tic_df['close'].to_numpy()  # Extract close prices
        
#         # Perform VMD on the close prices
#         u, _, _ = VMD(close_prices, alpha, tau, K, DC, init, tol)
        
#         # Check if the lengths match, and if not, align them
#         if u.shape[1] != len(tic_df):
#             print(f"Length mismatch for {tic}: VMD output has {u.shape[1]} points, expected {len(tic_df)}")
#             min_len = min(u.shape[1], len(tic_df))
#             tic_df = tic_df.iloc[:min_len]  # Truncate the dataframe to match VMD output length
#             u = u[:, :min_len]  # Truncate VMD output if necessary
        
#         # Assign VMD modes to columns
#         for k in range(K):
#             tic_df[f'vmd_mode_{k}'] = u[k, :]
        
#         # Append the dataframe with VMD modes
#         vmd_features.append(tic_df)
    
#     # Concatenate all VMD modes into one dataframe
#     vmd_df = pd.concat(vmd_features, axis=0)
    
#     return vmd_df
# #apply VMD separately to each train, 2021, 2022, 2023 data
# df_portfolio_comb_5 = apply_vmd_to_ticker(df_portfolio_comb_4)

In [10]:
from vmdpy import VMD
import pandas as pd
import numpy as np

def apply_vmd_to_ticker_rolling(df, alpha=5000, tau=0, K=3, DC=0, init=1, tol=1e-7, window=365):
    """
    Applies VMD to the 'close' price for each ticker in the dataframe using a rolling window approach.
    This ensures that each VMD decomposition is based only on the previous `window` ticks, avoiding forward leakage.
    
    Args:
        df (pd.DataFrame): Dataframe containing 'close' price data and 'tic' column.
        alpha, tau, K, DC, init, tol: Parameters for the VMD decomposition.
        window (int): Number of previous ticks to use for the rolling VMD.

    Returns:
        pd.DataFrame: DataFrame with the VMD modes added for each ticker.
    """
    vmd_features = []
    
    # Apply VMD for each ticker
    for tic in df['tic'].unique():
        tic_df = df[df['tic'] == tic].copy()  # Filter data for the current ticker
        close_prices = tic_df['close'].to_numpy()  # Extract close prices
        
        # Initialize empty columns for VMD modes
        for k in range(K):
            tic_df[f'vmd_mode_{k}'] = np.nan  # Initialize with NaNs

        # Apply VMD in a rolling window manner
        for i in range(window, len(close_prices)):
            close_window = close_prices[i - window:i]  # Select the previous `window` close prices
            
            # Perform VMD on the window of close prices
            u, _, _ = VMD(close_window, alpha, tau, K, DC, init, tol)
            
            # Assign the latest VMD modes (from the last tick of the window) to the current row
            for k in range(K):
                tic_df.at[tic_df.index[i], f'vmd_mode_{k}'] = u[k, -1]  # Assign last mode from the VMD result
        
        # Append the dataframe with VMD modes
        vmd_features.append(tic_df)
    
    # Concatenate all VMD modes into one dataframe
    vmd_df = pd.concat(vmd_features, axis=0)
    
    return vmd_df

df_portfolio_comb_5 = apply_vmd_to_ticker_rolling(df_portfolio_comb_4)


In [11]:
df_portfolio_final = df_portfolio_comb_5

In [12]:
# Check the data types of all columns in the DataFrame
print(df_portfolio_final.dtypes)

date                              datetime64[ns]
open                                     float64
high                                     float64
low                                      float64
close                                    float64
volume                                     int64
tic                                       object
day                                        int32
Open_VIX                                 float64
High_VIX                                 float64
Low_VIX                                  float64
Close_VIX                                float64
Adj Close_VIX                            float64
Open_SPY                                 float64
High_SPY                                 float64
Low_SPY                                  float64
Close_SPY                                float64
Adj Close_SPY                            float64
Volume_SPY                                 int64
Fear Greed                               float64
ratingScore         

In [13]:
#Convert Str to Float64
df_portfolio_final['net_export_goods_and_services'] = df_portfolio_final['net_export_goods_and_services'].str.replace(',', '').astype('float64')
df_portfolio_final['expenses'] = df_portfolio_final['expenses'].str.replace(',', '').astype('float64')
df_portfolio_final['exports'] = df_portfolio_final['exports'].str.replace(',', '').astype('float64')
df_portfolio_final['exports_goods'] = df_portfolio_final['exports_goods'].str.replace(',', '').astype('float64')
df_portfolio_final['exports_services'] = df_portfolio_final['exports_services'].str.replace(',', '').astype('float64')
df_portfolio_final['imports'] = df_portfolio_final['imports'].str.replace(',', '').astype('float64')
df_portfolio_final['imports_goods'] = df_portfolio_final['imports_goods'].str.replace(',', '').astype('float64')
df_portfolio_final['imports_services'] = df_portfolio_final['imports_services'].str.replace(',', '').astype('float64')

#Convert rest to Float64
df_portfolio_final[df_portfolio_final.columns.difference(['date', 'tic'])] = df_portfolio_final[df_portfolio_final.columns.difference(['date', 'tic'])].astype('float64')

#Convert date back
df_portfolio_final['date'] = df_portfolio_final['date'].dt.strftime('%Y-%m-%d')

In [14]:
# Filling NaN values with 0
df_portfolio_final.fillna(0, inplace=True)

In [15]:
portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_final)
portfolio_norm_df

,date,open,high,low,close,volume,tic,day,Open_VIX,High_VIX,...,TTM Sales per Share,Price to Sales Ratio,Assets,NetIncomeLoss,TTM Net Income (Billions),Shareholder's Equity (Billion),Return on Equity(%),vmd_mode_0,vmd_mode_1,vmd_mode_2
0,2011-01-03,0.058731,0.059087,0.058891,0.050363,0.236650,AAPL,0.00,0.216955,0.210015,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
10,2011-01-04,0.059958,0.059488,0.059491,0.050626,0.164317,AAPL,0.25,0.209699,0.213408,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
20,2011-01-05,0.059437,0.059817,0.059735,0.051040,0.135843,AAPL,0.50,0.215383,0.210015,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
30,2011-01-06,0.060369,0.059980,0.060352,0.050999,0.159718,AAPL,0.75,0.203168,0.205452,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
40,2011-01-07,0.060237,0.060177,0.060170,0.051364,0.165833,AAPL,1.00,0.209336,0.211419,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32659,2023-12-22,0.852725,0.852858,0.861494,0.855590,0.152474,XOM,1.00,0.165921,0.163332,...,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644,0.890019,-0.016307,-0.045132
32669,2023-12-26,0.856309,0.853604,0.864032,0.857521,0.198649,XOM,0.25,0.166526,0.161460,...,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644,0.889950,-0.013535,0.042860
32679,2023-12-27,0.850475,0.849627,0.857433,0.853491,0.171790,XOM,0.50,0.157456,0.152568,...,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644,0.890017,0.003982,-0.009066
32689,2023-12-28,0.845057,0.841839,0.847195,0.841150,0.192681,XOM,0.75,0.150441,0.148005,...,0.0,0.253363,0.963788,0.561798,0.0,0.0,0.807644,0.890248,0.016329,-0.029956


In [35]:
df_portfolio = portfolio_norm_df[["date", 
                                  "tic", 
                                  "close",
                                  "vmd_mode_0",
                                  "vmd_mode_1",
                                  "vmd_mode_2",
                                  "high", 
                                  "low",
                                  'High_VIX',
                                  'Low_VIX',
                                  'Close_VIX',
                                  'High_SPY',
                                  'Low_SPY',
                                  'Close_SPY',
                                  'Volume_SPY',
                                  'Fear Greed', 
                                  'ratingScore',
                                  'ratingDetailsDCFScore',
                                  'ratingDetailsROEScore',
                                  'ratingDetailsROAScore',
                                  'ratingDetailsPEScore',
                                  'ratingDetailsPBScore',
                                  'expenses',
                                  'GDP', 
                                  'exports', 
                                  'imports',
                                  'pound_dollar_exchange_rate', 
                                  'unemployment',
                                  'ics',
                                  'TTM Revenue(Billion)',
                                  'TTM EBITDA(Billion)',
                                  'EBITDA Margin',
                                  'Price to Book Ratio',
                                  'PE Ratio',
                                  'Price to Sales Ratio',
                                  'Assets',
                                  'NetIncomeLoss',
                                  'Return on Equity(%)']]

In [8]:
# df_portfolio_train = df_portfolio[(df_portfolio["date"] >= START_DATE) & (df_portfolio["date"] < "2020-12-31")]
# df_portfolio_test = df_portfolio[(df_portfolio["date"] >= "2021-01-01") & (df_portfolio["date"] < "2022-12-31")]


df_portfolio = pd.read_parquet('datasets/df_portfolio_train.parquet')
df_portfolio_test = pd.read_parquet('datasets/df_portfolio_test.parquet')

df_portfolio_train = df_portfolio[df_portfolio['date'] <= "2020-12-31"]
df_portfolio_vali = df_portfolio[df_portfolio['date'] >= "2021-01-01"]

# Feature Selection and DRL Model Parameters

In [10]:
import pandas as pd
from itertools import product
import numpy as np  # for NaN
import torch

FEATURE_NAMES = ["close",
               "vmd_mode_0",
                "vmd_mode_1",
                "vmd_mode_2",
                  #"high", 
                  #"low",
                'High_VIX',
                'Low_VIX',
                  #'Close_VIX',
                'High_SPY',
                'Low_SPY',
                'Close_SPY',
                  #'Volume_SPY',
                'Fear Greed', 
                'ratingScore',
                  #'ratingDetailsDCFScore',
                'ratingDetailsROEScore',
                  #'ratingDetailsROAScore',
                  #'ratingDetailsPEScore',
                  #'ratingDetailsPBScore',
                  #'expenses',
                  #'GDP', 
                'exports', 
                  #'imports',
                'pound_dollar_exchange_rate', 
                'unemployment',
                'ics',
                'TTM Revenue(Billion)',
                  #'TTM EBITDA(Billion)',
                  #'EBITDA Margin',
                'Price to Book Ratio',
                  #'PE Ratio',
                'Price to Sales Ratio',
                  #'Assets',
                  #'NetIncomeLoss',
                'Return on Equity(%)']

COMISSION_FEE = 0.001

# Define hyperparameter ranges
time_windows = [60]
k_sizes = [3, 4, 5]
conv_mids = [5 , 10, 20]
conv_finals = [5, 10, 20]
lr = [0.02]

# Function to calculate the final accumulative portfolio value
def calculate_cumulative_value(portfolio_values):
    initial_value = portfolio_values[0]
    final_value = portfolio_values[-1]
    return final_value / initial_value

# Function to calculate the maximum drawdown
def calculate_max_drawdown(portfolio_values):
    portfolio_values = np.array(portfolio_values)
    running_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (portfolio_values - running_max) / running_max
    max_drawdown = drawdowns.min()
    
    return max_drawdown

# Function to calculate the Sharpe ratio
def calculate_sharpe_ratio(portfolio_values, risk_free_rate=0):
    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    excess_returns = returns - risk_free_rate
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    sharpe_ratio_annualized = sharpe_ratio * np.sqrt(252)
    
    return sharpe_ratio_annualized


# Define a function to calculate the metrics (this is just a placeholder; replace with actual functions)
def calculate_metrics(environment):
    final_portfolio_value = environment._asset_memory["final"][-1]
    max_drawdown = calculate_max_drawdown(environment._asset_memory["final"])
    sharpe_ratio = calculate_sharpe_ratio(environment._asset_memory["final"])
    cumulative_value = calculate_cumulative_value(environment._asset_memory["final"])
    return final_portfolio_value, max_drawdown, sharpe_ratio, cumulative_value

# DataFrame to store results
columns = ["TIME_WINDOW", "K_SIZE", "CONV_MID", "CONV_FINAL","LEARNING_RATE", 
        'train_portfolio_value', 'train_drawdown', 'train_sharpe', 'train_cumulative',
        'test_portfolio_value', 'test_drawdown', 'test_sharpe', 'test_cumulative']

results_df = pd.DataFrame(columns=columns)


In [12]:
# Hyperparameter tuning loop with 5 episodes of training and error handling
for TIME_WINDOW, K_SIZE, CONV_MID, CONV_FINAL,LEARNING_RATE in product(time_windows, k_sizes, conv_mids, conv_finals, lr):
    try:
        print(f"Training model with TIME_WINDOW={TIME_WINDOW}, K_SIZE={K_SIZE}, CONV_MID={CONV_MID}, CONV_FINAL={CONV_FINAL}, LEARNING_RATE={LEARNING_RATE}")
        # Update the policy_kwargs for the current set of hyperparameters
        policy_kwargs = {
        "initial_features": len(FEATURE_NAMES),
        "k_size": K_SIZE,
        "time_window": TIME_WINDOW,
        "conv_mid_features":CONV_MID,
        "conv_final_features":CONV_FINAL}

        # Create the environment for training and the test years
        environment = PortfolioOptimizationEnv(df_portfolio_train, initial_amount=100000, comission_fee_pct=COMISSION_FEE,
                                               time_window=TIME_WINDOW, features=FEATURE_NAMES, normalize_df=None,
                                               reward_function=custom_reward_function, reward_scaling=2.0)

        environment_test = PortfolioOptimizationEnv(df_portfolio_vali, initial_amount=100000, comission_fee_pct=COMISSION_FEE,
                                                    time_window=TIME_WINDOW, features=FEATURE_NAMES, normalize_df=None,
                                                    reward_function=custom_reward_function, reward_scaling=2.0)

        # Instantiate the model for each combination of hyperparameters
        # set PolicyGradient parameters
        model_kwargs = {
                "lr": LEARNING_RATE,
                "policy": EIIE,
                }

        EIIE_model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)
        EIIE_policy = EIIE(time_window=TIME_WINDOW, device=device, initial_features=len(FEATURE_NAMES),
                           k_size=K_SIZE, conv_mid_features=CONV_MID, conv_final_features=CONV_FINAL)

        # Train the model for 10 episodes
        DRLAgent.train_model(EIIE_model, episodes=25)
        
        # Save the model policy after training
        model_filename = f"policy_EIIE_{TIME_WINDOW}_{K_SIZE}_{CONV_MID}_{CONV_FINAL}_{LEARNING_RATE}.pt"
        torch.save(EIIE_model.train_policy.state_dict(), model_filename)
        
        EIIE_policy = EIIE(time_window = TIME_WINDOW, device = device, initial_features = len(FEATURE_NAMES),
                          k_size = K_SIZE, conv_mid_features = CONV_MID,conv_final_features = CONV_FINAL)

        EIIE_policy.load_state_dict(torch.load(model_filename))

        # Training evaluation
        environment.reset()
        DRLAgent.DRL_validation(EIIE_model, environment, policy=EIIE_policy)
        train_final_value, train_drawdown, train_sharpe, train_cumulative = calculate_metrics(environment)

        # Test evaluation
        environment_test.reset()
        DRLAgent.DRL_validation(EIIE_model, environment_test, policy=EIIE_policy)
        test_final_value, test_drawdown, test_sharpe, test_cumulative = calculate_metrics(environment_test)

 

    except Exception as e:
        print(f"Error encountered with combination policy_EIIE_{TIME_WINDOW}_{K_SIZE}_{CONV_MID}_{CONV_FINAL}: {e}")
        # If an error occurs, store NaN for all the metrics
        train_final_value = train_drawdown = train_sharpe = train_cumulative = np.nan
        test_final_value = test_drawdown = test_sharpe = test_cumulative = np.nan
 

    # Create a temporary DataFrame to store the current results
    temp_df = pd.DataFrame([{
        "TIME_WINDOW": TIME_WINDOW, "K_SIZE": K_SIZE, "CONV_MID": CONV_MID, "CONV_FINAL": CONV_FINAL, "LEARNING_RATE": LEARNING_RATE,
        "train_portfolio_value": train_final_value, "train_drawdown": train_drawdown, "train_sharpe": train_sharpe, "train_cumulative": train_cumulative,
        "test_portfolio_value": test_final_value, "test_drawdown": test_drawdown, "test_sharpe": test_sharpe, "test_cumulative": test_cumulative,
        }])
    
    # Use pd.concat instead of append to add the results to the main DataFrame
    results_df = pd.concat([results_df, temp_df], ignore_index=True)
    
    try:
        results_df.to_csv('hyperparamstuning60.csv', index=False)
    except Exception as e:
        print(f"An error occurred: {e}")
        
    
results_df

Training model with TIME_WINDOW=60, K_SIZE=3, CONV_MID=5, CONV_FINAL=5, LEARNING_RATE=0.02


  0%|          | 0/25 [00:00<?, ?it/s]

Initial portfolio value: 100000
Final portfolio value: 225579.046875
Final accumulative portfolio return: 2.25579046875
Maximum DrawDown: -0.33093055752335365
Sharpe ratio: 0.7694429040925047


  4%|▍         | 1/25 [00:27<11:04, 27.67s/it]

Initial portfolio value: 100000
Final portfolio value: 228228.203125
Final accumulative portfolio return: 2.28228203125
Maximum DrawDown: -0.3349048176962024
Sharpe ratio: 0.7702749987377095


...

 96%|█████████▌| 24/25 [11:07<00:27, 27.62s/it]

Initial portfolio value: 100000
Final portfolio value: 702046.6875
Final accumulative portfolio return: 7.020466875
Maximum DrawDown: -0.44789773427946766
Sharpe ratio: 1.0912489464202453


100%|██████████| 25/25 [11:35<00:00, 27.80s/it]
C:\Users\xutia\AppData\Local\Temp\ipykernel_36848\1259870935.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  EIIE_policy

Initial portfolio value: 100000
Final portfolio value: 621644.0625
Final accumulative portfolio return: 6.216440625
Maximum DrawDown: -0.30790142231335127
Sharpe ratio: 1.1139706135231697
Initial portfolio value: 100000
Final portfolio value: 200760.015625
Final accumulative portfolio return: 2.00760015625
Maximum DrawDown: -0.1961074257155252
Sharpe ratio: 1.4982962749027082


,TIME_WINDOW,K_SIZE,CONV_MID,CONV_FINAL,LEARNING_RATE,train_portfolio_value,train_drawdown,train_sharpe,train_cumulative,test_portfolio_value,test_drawdown,test_sharpe,test_cumulative
0,60,3,5,5,0.02,2.380566e+05,-0.351223,0.771384,2.380566,126201.554688,-0.158603,0.842385,1.262016
1,60,3,5,10,0.02,3.693064e+05,-0.342045,0.818966,3.693064,85508.687500,-0.388765,-0.167985,0.855087
2,60,3,5,20,0.02,4.200843e+05,-0.347251,0.908278,4.200843,97444.843750,-0.339930,0.092566,0.974448
3,60,3,10,5,0.02,2.380582e+05,-0.351224,0.771384,2.380582,126201.375000,-0.158605,0.842375,1.262014
4,60,3,10,10,0.02,1.000000e+05,0.000000,NaN,1.000000,100000.000000,0.000000,NaN,1.000000
5,60,3,10,20,0.02,6.553672e+05,-0.328612,1.265267,6.553673,92322.421875,-0.300691,-0.025217,0.923224
6,60,3,20,5,0.02,2.620481e+05,-0.456307,0.622341,2.620481,171522.500000,-0.217306,1.215183,1.715225
7,60,3,20,10,0.02,1.018255e+06,-0.287279,1.379930,10.182549,133215.843750,-0.229391,0.712847,1.332158
8,60,3,20,20,0.02,6.385760e+05,-0.385159,1.117825,6.385760,132942.281250,-0.331567,0.680240,1.329423
9,60,4,5,5,0.02,6.444928e+05,-0.322589,1.133857,6.444928,54166.011719,-0.589159,-0.982752,0.541660


In [ ]:
# =(M2-MIN($M$2:$M$163))/((MAX($M$2:$M$163)-MIN($M$2:$M$163)))
# =(L2-MIN($L$2:$L$163))/((MAX($L$2:$L$163)-MIN($L$2:$L$163)))
# =(K2-MIN($K$2:$K$163))/((MAX($K$2:$K$163)-MIN($K$2:$K$163)))
# =(N2+O2+P2)/3

In [7]:
results_df

,TIME_WINDOW,K_SIZE,CONV_MID,CONV_FINAL,LEARNING_RATE,train_portfolio_value,train_drawdown,train_sharpe,train_cumulative,test_portfolio_value,test_drawdown,test_sharpe,test_cumulative
0,60,3,5,5,0.02,669630.37500,-0.323755,1.162123,6.696304,58291.769531,-0.468099,-1.035075,0.582918
1,60,3,5,10,0.02,401553.78125,-0.307901,0.935154,4.015538,201765.078125,-0.202645,1.477121,2.017651
2,60,3,10,5,0.02,851647.00000,-0.385159,1.276073,8.516470,205044.046875,-0.216669,1.483678,2.050440
3,60,3,10,10,0.02,800436.93750,-0.333297,1.279539,8.004369,210115.593750,-0.206187,1.644595,2.101156
4,60,3,20,5,0.02,795419.31250,-0.385159,1.216382,7.954193,138230.015625,-0.272178,0.764831,1.382300
5,60,3,20,10,0.02,757926.06250,-0.385159,1.209615,7.579261,81028.585938,-0.342613,-0.331577,0.810286
6,60,4,5,5,0.02,235260.15625,-0.347388,0.770433,2.352602,125748.773438,-0.156451,0.840368,1.257488
7,60,4,5,10,0.02,614110.93750,-0.385159,1.115737,6.141109,195518.640625,-0.214764,1.401475,1.955186
8,60,4,10,5,0.02,687353.68750,-0.280393,1.209436,6.873537,174014.312500,-0.205600,1.243291,1.740143
9,60,4,10,10,0.02,340560.84375,-0.296457,0.860082,3.405608,95031.023438,-0.306015,0.038745,0.950310


In [ ]:
results_df2.to_csv('results_df2.csv', index=False)

In [35]:
results_df.to_csv('secondhyperparmatuning.csv', index=False)

In [36]:
FEATURE_NAMES = ['close',
                 'high',
                 'low',
                 'High_VIX',
                 'Low_VIX',
                 'Close_VIX',
                 'High_SPY',
                 'Low_SPY',
                 'Close_SPY',
                 'Volume_SPY',
                 'Fear Greed', 
                 'ratingScore',
                 'ratingDetailsDCFScore',
                 'ratingDetailsROEScore',
                 'ratingDetailsROAScore',
                 'ratingDetailsPEScore',
                 'ratingDetailsPBScore',
                 'expenses',
                 'GDP', 
                 'exports', 
                 'imports',
                 'pound_dollar_exchange_rate',
                 'unemployment',
                 'ics',
                 'TTM Revenue(Billion)',
                 'TTM EBITDA(Billion)',
                 'EBITDA Margin',
                 'Price to Book Ratio',
                 'PE Ratio',
                 'Price to Sales Ratio',
                 'Assets',
                 'NetIncomeLoss',
                 'Return on Equity(%)']

TIME_WINDOW = 50
COMISSION_FEE = 0.0025
K_SIZE = 4
CONV_MID = 5
CONV_FINAL= 20

environment = PortfolioOptimizationEnv(
        df_portfolio_train,
        initial_amount=100000,
        comission_fee_pct=COMISSION_FEE,
        time_window=TIME_WINDOW,
        features=FEATURE_NAMES,
        normalize_df=None,
        reward_function=custom_reward_function,
        reward_scaling=1.0
    )

# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.01, 
    "policy": EIIE,
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "initial_features": len(FEATURE_NAMES),
    "k_size": K_SIZE,
    "time_window": TIME_WINDOW,
    "conv_mid_features":CONV_MID,
    "conv_final_features":CONV_FINAL}

EIIE_model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)

# DRL Model Training (Jan 2011- Dec 2020)

In [ ]:
DRLAgent.train_model(EIIE_model, episodes=5)
torch.save(EIIE_model.train_policy.state_dict(), "policy_EIIE(test).pt")

# DRL Model Evaluation (Jan 2021-Dec 2023)

In [ ]:
import os
import shutil

environment_2021 = PortfolioOptimizationEnv(
    df_portfolio_2021,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

environment_2022 = PortfolioOptimizationEnv(
    df_portfolio_2022,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

environment_2023 = PortfolioOptimizationEnv(
    df_portfolio_2023,
    initial_amount=100000,
    comission_fee_pct=COMISSION_FEE,
    time_window=TIME_WINDOW,
    features=FEATURE_NAMES,
    normalize_df=None,
    reward_function=custom_reward_function,
    reward_scaling=1.0
)

EIIE_results = {
    "train": {},
    "2021": {},
    "2022": {},
    "2023": {}
}
# EI3_results = {
#     "train": {},
#     "2021": {},
#     "2022": {},
#     "2023": {}
# }

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.

EIIE_policy = EIIE(time_window = TIME_WINDOW, device = device, initial_features = len(FEATURE_NAMES),
              k_size = K_SIZE, conv_mid_features = CONV_MID,conv_final_features = CONV_FINAL)

EIIE_policy.load_state_dict(torch.load("policy_EIIE(test).pt"))

environment.reset()
src_folder = "results/rl"
dst_folder = "results/train"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["train"]["value"] = environment._asset_memory["final"]
EIIE_results["train"]["actions"] = environment._actions_memory

src_folder = "results/rl"
dst_folder = "results/2021"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2021, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2021"]["value"] = environment_2021._asset_memory["final"]
EIIE_results["2021"]["actions"] = environment_2021._actions_memory

src_folder = "results/rl"
dst_folder = "results/2022"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2022, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2022"]["value"] = environment_2022._asset_memory["final"]
EIIE_results["2022"]["actions"] = environment_2022._actions_memory

src_folder = "results/rl"
dst_folder = "results/2023"
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
DRLAgent.DRL_validation(EIIE_model, environment_2023, policy=EIIE_policy)
os.rename(src_folder, dst_folder)
EIIE_results["2023"]["value"] = environment_2023._asset_memory["final"]
EIIE_results["2023"]["actions"] = environment_2023._actions_memory


# EI3_policy = EI3(time_window=50, device=device)
# EI3_policy.load_state_dict(torch.load("policy_EI3.pt"))

# environment.reset()
# DRLAgent.DRL_validation(EI3_model, environment, policy=EI3_policy)
# EI3_results["train"]["value"] = environment._asset_memory["final"]
# environment_2021.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2021, policy=EI3_policy)
# EI3_results["2021"]["value"] = environment_2021._asset_memory["final"]
# environment_2022.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2022, policy=EI3_policy)
# EI3_results["2022"]["value"] = environment_2022._asset_memory["final"]
# environment_2023.reset()
# DRLAgent.DRL_validation(EI3_model, environment_2023, policy=EI3_policy)
# EI3_results["2023"]["value"] = environment_2023._asset_memory["final"]

#uniform buy n hold
if not os.path.exists(src_folder):
    os.makedirs(src_folder)
UBAH_results = {
    "train": {"value": [], "actions": []},
    "2021": {"value": [], "actions": []},
    "2022": {"value": [], "actions": []},
    "2023": {"value": [], "actions": []}
}

PORTFOLIO_SIZE = len(TEST_SET)

# train period
terminated = False
environment.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment.step(action)
    UBAH_results["train"]["actions"].append(action)
UBAH_results["train"]["value"] = environment._asset_memory["final"]

# 2021
terminated = False
environment_2021.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2021.step(action)
    UBAH_results["2021"]["actions"].append(action)
UBAH_results["2021"]["value"] = environment_2021._asset_memory["final"]

# 2022
terminated = False
environment_2022.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2022.step(action)
    UBAH_results["2022"]["actions"].append(action)
UBAH_results["2022"]["value"] = environment_2022._asset_memory["final"]

# 2023
terminated = False
environment_2023.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_2023.step(action)
    UBAH_results["2023"]["actions"].append(action)
UBAH_results["2023"]["value"] = environment_2023._asset_memory["final"]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(UBAH_results["train"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["train"]["value"], label="EIIE")
#plt.plot(EI3_results["train"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in training period")
plt.legend()

plt.show()

plt.plot(UBAH_results["2021"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2021"]["value"], label="EIIE")
#plt.plot(EI3_results["2021"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2021")
plt.legend()

plt.show()

plt.plot(UBAH_results["2022"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2022"]["value"], label="EIIE")
#plt.plot(EI3_results["2022"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2022")
plt.legend()

plt.show()

plt.plot(UBAH_results["2023"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["2023"]["value"], label="EIIE")
#plt.plot(EI3_results["2023"]["value"], label="EI3")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in 2023")
plt.legend()

plt.show()

# Portfolio Allocation by Trained DRL Model

In [ ]:
# Function to plot action memory
def plot_action_memory(actions, tickers, label):
    """Visualize portfolio allocation over time."""
    actions_df = pd.DataFrame(actions, columns=["cash"] + tickers)
    actions_df.plot(kind="line", figsize=(12, 8), title=f"Portfolio Allocation Breakdown - {label}")
    plt.ylabel("Portfolio Weight")
    plt.xlabel("Time Step")
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot EIIE actions for the training period
plot_action_memory(EIIE_results["train"]["actions"], TEST_SET, "EIIE - Train")

# Plot EIIE actions for 2021
plot_action_memory(EIIE_results["2021"]["actions"], TEST_SET, "EIIE - 2021")

# Plot EIIE actions for 2022
plot_action_memory(EIIE_results["2022"]["actions"], TEST_SET, "EIIE - 2022")

# Plot EIIE actions for 2023
plot_action_memory(EIIE_results["2023"]["actions"], TEST_SET, "EIIE - 2023")

# Similarly, plot UBAH actions for each period
# plot_action_memory(UBAH_results["train"]["actions"], TEST_SET, "UBAH - Train")
# plot_action_memory(UBAH_results["2021"]["actions"], TEST_SET, "UBAH - 2021")
# plot_action_memory(UBAH_results["2022"]["actions"], TEST_SET, "UBAH - 2022")
# plot_action_memory(UBAH_results["2023"]["actions"], TEST_SET, "UBAH - 2023")

In [ ]:
results_df2 = results_df2[["TIME_WINDOW", "K_SIZE", "CONV_MID", "CONV_FINAL", 
            'train_portfolio_value', 'train_drawdown', 'train_sharpe','train_cumulative',
            '2021_portfolio_value', '2021_drawdown', '2021_sharpe','2021_cumulative',
             '2022_portfolio_value', '2022_drawdown', '2022_sharpe','2022_cumulative',
             '2023_portfolio_value', '2023_drawdown', '2023_sharpe','2023_cumulative']]